In [2]:
import pandas as pd
import os
import json
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.style.use('classic')


In [3]:
if os.getcwd().split('/')[-1] != 'market-congestion':
    os.chdir('..')

print(f'working directory: {os.getcwd()}')


working directory: /Users/crinstaniev/Dev/market-congestion


In [4]:
timestamp_file = open('data/blocks_timestamp.json', 'r')
timestamp_list = json.loads(timestamp_file.read())
timestamp_df = pd.DataFrame(timestamp_list)

display(timestamp_df.head())


,date,block,timestamp
0,2022-04-25T00:00:00Z,14650515,1650844804
1,2022-04-25T01:00:00Z,14650789,1650848411
2,2022-04-25T02:00:00Z,14651041,1650852008
3,2022-04-25T03:00:00Z,14651321,1650855605
4,2022-04-25T04:00:00Z,14651576,1650859213


In [5]:
block_max = timestamp_df['block'].max()
block_min = timestamp_df['block'].min()

print(f'block range: {block_min} to {block_max}')


block range: 14650515 to 14713964


In [6]:
fetch_blocks_command = f'ethereumetl export_blocks_and_transactions\
        --start-block {block_min}\
        --end-block {block_max}\
        --blocks-output data/blocks.csv\
        --provider-uri https://mainnet.infura.io/v3/b5502deb425f4629a1c886601e332e56'

# we collect data during the drop
fetch_blocks_command = [x for x in fetch_blocks_command.split(' ') if x != '']


In [7]:
# subprocess.check_output(fetch_blocks_command)


preprocess blocks data

In [71]:
blocks_df = pd.read_csv('data/blocks.csv')
# exclude object types
blocks_df = blocks_df.select_dtypes(
    exclude=['object']).sort_values(by='number')
blocks_df['timestamp'] = pd.to_datetime(blocks_df['timestamp'], unit='s')
blocks_df.head()


,number,difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas
100,14650515,13689034192123421,543,30000000,0,2022-04-25 00:00:04,0,24352688619
101,14650516,13695855729803515,11865,29970705,3067277,2022-04-25 00:00:07,37,21308602542
102,14650517,13702680598312555,293835,29941438,29927116,2022-04-25 00:00:09,310,19190221179
103,14650518,13696127275255133,146476,29970676,29951281,2022-04-25 00:00:35,224,21586703993
104,14650519,13702952276354725,79858,29999943,15598681,2022-04-25 00:00:38,251,24281549627


In [72]:
blocks_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 63450 entries, 100 to 63249
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   number             63450 non-null  int64         
 1   difficulty         63450 non-null  int64         
 2   size               63450 non-null  int64         
 3   gas_limit          63450 non-null  int64         
 4   gas_used           63450 non-null  int64         
 5   timestamp          63450 non-null  datetime64[ns]
 6   transaction_count  63450 non-null  int64         
 7   base_fee_per_gas   63450 non-null  int64         
dtypes: datetime64[ns](1), int64(7)
memory usage: 4.4 MB


In [73]:
CONGESTION_THRESHOLD = 0.95

blocks_df['theta'] = blocks_df['gas_used'] / blocks_df['gas_limit']
blocks_df['congested'] = blocks_df['theta'] >= CONGESTION_THRESHOLD
# blocks_df['timestamp'] = pd.to_datetime(blocks_df['timestamp'])

blocks_df


,number,difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,theta,congested
100,14650515,13689034192123421,543,30000000,0,2022-04-25 00:00:04,0,24352688619,0.000000,False
101,14650516,13695855729803515,11865,29970705,3067277,2022-04-25 00:00:07,37,21308602542,0.102343,False
102,14650517,13702680598312555,293835,29941438,29927116,2022-04-25 00:00:09,310,19190221179,0.999522,True
103,14650518,13696127275255133,146476,29970676,29951281,2022-04-25 00:00:35,224,21586703993,0.999353,True
104,14650519,13702952276354725,79858,29999943,15598681,2022-04-25 00:00:38,251,24281549627,0.519957,False
...,...,...,...,...,...,...,...,...,...,...
63245,14713960,13978067652195229,64125,30000000,6487887,2022-05-04 23:59:15,74,56254763659,0.216263,False
63246,14713961,13985167758447971,13029,30000000,1785845,2022-05-04 23:59:22,30,52264372784,0.059528,False
63247,14713962,13985442636354915,62044,30000000,12974311,2022-05-04 23:59:37,203,46509126760,0.432477,False
63248,14713963,13985717514261859,150103,30000000,27986215,2022-05-04 23:59:53,401,45724018207,0.932874,False


train test split

In [74]:
from sklearn.model_selection import TimeSeriesSplit


In [75]:
tss = TimeSeriesSplit()
for train_idx, test_idx in tss.split(blocks_df):
    pass

train_df = blocks_df.filter(items=train_idx, axis=0)
test_df = blocks_df.filter(items=test_idx, axis=0)


In [76]:
MA_WINDOW = 100


In [77]:
def prepare_ma_prev(df, window):
    return df['theta'].rolling(window=window).mean()


In [78]:
train_df['theta_ma_prev'] = prepare_ma_prev(train_df, MA_WINDOW)
train_df.dropna(inplace=True)

test_df['theta_ma_prev'] = prepare_ma_prev(test_df, MA_WINDOW)
test_df.dropna(inplace=True)


In [79]:
train_df.to_pickle('data/train.pkl')
test_df.to_pickle('data/test.pkl')


## Discrete time series data

In [80]:
train_df_clean = train_df[[
    'size', 'transaction_count', 'theta', 'theta_ma_prev', 'congested'
]]

test_df_clean = test_df[[
    'size', 'transaction_count', 'theta', 'theta_ma_prev', 'congested'
]]


In [81]:
ROLLING_WINDOW = 10


In [82]:
X_train_dis = []
y_train_dis = []

for i in tqdm(range(len(train_df_clean))):
    if i < ROLLING_WINDOW:
        continue

    X_train_dis.append(
        train_df_clean.iloc[i-ROLLING_WINDOW:i][['size', 'transaction_count', 'theta_ma_prev', 'congested', 'theta']].values.flatten())
    y_train_dis.append(train_df_clean.iloc[i])

X_train_dis = np.array(X_train_dis)
y_train_dis = pd.DataFrame(y_train_dis)


100%|██████████| 52776/52776 [00:19<00:00, 2763.08it/s]


In [83]:
X_test_dis = []
y_test_dis = []

for i in tqdm(range(len(test_df_clean))):
    if i < ROLLING_WINDOW:
        continue

    X_test_dis.append(test_df_clean.iloc[i-ROLLING_WINDOW:i][[
                      'size', 'transaction_count', 'theta', 'theta_ma_prev', 'congested']].values.flatten())
    y_test_dis.append(test_df_clean.iloc[i])

X_test_dis = np.array(X_test_dis)
y_test_dis = pd.DataFrame(y_test_dis)


100%|██████████| 10476/10476 [00:03<00:00, 3118.36it/s]


In [84]:
f = open('data/train_test_dis.pkl', 'wb')
train_test_dis = dict(
    X_train_dis=X_train_dis,
    y_train_dis=y_train_dis,
    X_test_dis=X_test_dis,
    y_test_dis=y_test_dis
)
pickle.dump(train_test_dis, f)


In [85]:
train_df['theta_ma_future'] = train_df['theta'].shift(1 - MA_WINDOW)\
    .rolling(window=MA_WINDOW).mean()
test_df['theta_ma_future'] = test_df['theta'].shift(1 - MA_WINDOW)\
    .rolling(window=MA_WINDOW).mean()


In [86]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)


In [87]:
train_df.to_pickle('data/regression_train_df.pkl')
test_df.to_pickle('data/regression_test_df.pkl')


## Introduce Volatility

In [88]:
prices_df = pd.read_csv(
    'data/price.csv')[['block_number', 'timestamp', 'price']]
prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'])

prices_df = prices_df.sort_values(by=['block_number', 'timestamp']
                                  ).groupby('block_number').last().reset_index()

prices_df.columns = ['number', 'timestamp', 'price']
prices_df = prices_df[['number', 'price']]
prices_df


,number,price
0,14650538,2922.818401
1,14650559,2919.021395
2,14650560,2914.937828
3,14650562,2909.986891
4,14650567,2908.532334
...,...,...
3089,14713680,2933.652317
3090,14713700,2933.652317
3091,14713702,2930.134222
3092,14713801,2930.134222


train dataset

In [89]:
train_df_with_price = train_df.merge(right=prices_df, how='left', on='number')
train_df_with_price['price'] = train_df_with_price['price'].interpolate(
    method='nearest')

train_df_with_price = train_df_with_price.dropna()
train_df_with_price['volatility'] = (
    np.log(train_df_with_price['price'] /
           train_df_with_price['price'].shift(1))
).rolling(window=MA_WINDOW).std()

train_df_with_price = train_df_with_price.dropna()
train_df_with_price


,number,difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,theta,congested,theta_ma_prev,theta_ma_future,price,volatility
101,14651014,13708625850173183,26836,30029295,5642122,2022-04-25 01:54:14,114,21313775011,0.187887,False,0.485427,0.529859,2860.937666,1.623016e-03
102,14651015,13715456954092559,99794,30000000,18066881,2022-04-25 01:54:16,161,19650699873,0.602229,False,0.481451,0.528910,2860.937666,1.623016e-03
103,14651016,13715594393046031,101022,30000000,10614851,2022-04-25 01:54:31,115,20152919523,0.353828,False,0.476527,0.531203,2862.654658,1.624684e-03
104,14651017,13722428899574232,88720,30000000,29983945,2022-04-25 01:54:36,322,19416473233,0.999465,True,0.483912,0.531066,2862.654658,1.624684e-03
105,14651018,13702465124319344,160108,30000000,27021111,2022-04-25 01:55:14,332,21840934624,0.900704,False,0.482921,0.522580,2862.654658,1.624684e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52425,14703338,13782730729074524,11018,30000000,3146343,2022-05-03 07:33:50,63,53402499370,0.104878,False,0.505523,0.509489,2838.710263,3.532075e-11
52426,14703339,13789735455970273,129581,29970705,29963121,2022-05-03 07:33:55,370,48127375117,0.999747,True,0.509442,0.513549,2838.710263,3.532075e-11
52427,14703340,13763077256814777,90356,29999972,20575759,2022-05-03 07:34:40,257,54140252784,0.685859,False,0.514546,0.508182,2838.710263,3.532075e-11
52428,14703341,13770072387288525,128520,30000000,29828994,2022-05-03 07:34:41,301,56655869795,0.994300,True,0.520033,0.503705,2838.710263,3.532075e-11


test dataset

In [90]:
test_df_with_price = test_df.merge(right=prices_df, how='left', on='number')
test_df_with_price['price'] = test_df_with_price['price'].interpolate(
    method='nearest')

test_df_with_price = test_df_with_price.dropna()
test_df_with_price['volatility'] = (
    np.log(test_df_with_price['price'] /
           test_df_with_price['price'].shift(1))
).rolling(window=MA_WINDOW).std()

test_df_with_price = test_df_with_price.dropna()
test_df_with_price


,number,difficulty,size,gas_limit,gas_used,timestamp,transaction_count,base_fee_per_gas,theta,congested,theta_ma_prev,theta_ma_future,price,volatility
139,14703827,13935170070759845,38833,29941351,6824583,2022-05-03 09:21:39,88,63806120676,0.227932,False,0.521236,0.492136,2844.961988,0.000126
140,14703828,13935444948666789,47582,29970589,9242226,2022-05-03 09:21:54,98,59466215084,0.308377,False,0.514362,0.491159,2844.961988,0.000040
141,14703829,13942524243052574,15423,29999856,3276874,2022-05-03 09:22:02,50,56617434499,0.109230,False,0.513033,0.495534,2844.961988,0.000040
142,14703830,13949606994125071,143575,30000000,29985365,2022-05-03 09:22:06,308,51086330934,0.999512,True,0.522119,0.498975,2844.961988,0.000040
143,14703831,13943070540491915,14063,30029295,2571355,2022-05-03 09:22:28,47,57465891896,0.085628,False,0.522388,0.491600,2844.961988,0.000040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10263,14713901,14037390798779093,139287,29970448,21219800,2022-05-04 23:44:22,277,34769527102,0.708024,False,0.522443,0.512474,2933.065675,0.000100
10264,14713902,14023957287234105,32664,29999715,5641542,2022-05-04 23:44:51,67,36577752150,0.188053,False,0.514339,0.508179,2933.065675,0.000100
10265,14713903,14031079800535206,550,29970420,0,2022-05-04 23:44:54,0,33725173901,0.000000,False,0.513986,0.516296,2933.065675,0.000100
10266,14713904,14038205791626005,144023,29941154,24229307,2022-05-04 23:44:56,243,29509527164,0.809231,False,0.512081,0.520790,2933.065675,0.000100


In [91]:
# save data
train_df_with_price.to_pickle('data/train_volatility.pkl')
test_df_with_price.to_pickle('data/test_volatility.pkl')
